### Problem Statement:
Given a sentence classify it to the one of the classes,'who' / 'what' / 'when' / 'affirmation' / 'unknown'

### Approach: 

1. Preprocess the data i.e., read the lines from the text file split and generate sentences. 
2. get the label for each of the sentence from the dataset. 
3. Segregate the sentences as the inputs (/contexts) and labels as the outputs

In [1]:
with open('nikkidata.txt') as f:
    text = f.readlines()
    sentences = []
    for i in text: sentences.append(i.split(" ,,, "))
context = []
labels = []
for sent in sentences:
    context.append(sent[0])
    labels.append(sent[1].split("\n")[0])

1. from the above cell, the context variable has all the sentences and the labels variable has all the classes respectively.
2. Each sentence is to be padded with a sentence start start and sentence end  token ( Not needed but I did to for a reason which I will explain it in below)
3. Once you have padded sentence, I used word_tokenize from nltk toolkit to tokenize the sentences 
4. the result at the end of pre-processing is displayed in the below cell.

In [2]:
import itertools
import nltk
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"
sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in context]
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]
print(tokenized_sentences[0], len(tokenized_sentences))

['SENTENCE_START', 'how', 'did', 'serfdom', 'develop', 'in', 'and', 'then', 'leave', 'russia', '?', 'SENTENCE_END'] 1483


From the above cell output, we see how a setences is tokenized into words. 

The next step is to see the word frequency of each word in the entire corpus.

In [3]:
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print("Found %d unique words tokens." % len(word_freq.items()))

Found 3716 unique words tokens.


nltk.FreqDist give the number of unique words that appear in the corpus. as we see, there are in total 3716 words that make up the all the sentences in the corpus. 

From the below, I am fixing the vocabulary size to 3500. This leaves us few sentences with missing words. In order to account for the missing words, we use an unknown token. ("UNKNOWN_TOKEN") 

So, whenever a word in a sentences is encountered to be missing from the vocabulary list, we fill the void with the unknown token

In [4]:
vocabulary_size = 3500
vocab = word_freq.most_common(vocabulary_size-1)

From the below cell, the example sentence and followed by the same sentence after pre-processing convey us the implementation of the concept I have explained in the above. 


In [5]:
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
print("Vocabulary size %d." % vocabulary_size)
print("\nThe least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1]))
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in index_to_word else unknown_token for w in sent]
print("\nExample sentence: '%s'" % sentences[0])
print("\nExample sentence after Pre-processing: '%s'" % tokenized_sentences[0])

Using vocabulary size 3500.
The least frequent word in our vocabulary is 'domesticated' and appeared 1 times.

Example sentence: 'SENTENCE_START how did serfdom develop in and then leave russia ? SENTENCE_END'

Example sentence after Pre-processing: '['SENTENCE_START', 'how', 'did', 'serfdom', 'develop', 'in', 'and', 'UNKNOWN_TOKEN', 'leave', 'russia', '?', 'SENTENCE_END']'


Here I am saving the tokenized sentences with pre-processing. I am doing this because, I am gonna use "GloVe" model to get the word embeddings for the words in the vocabulary list. 

In [6]:
import pickle
with open('data.txt', 'wb') as f:
    pickle.dump(tokenized_sentences, f)

The below cell, loads the words and embeddings obtained after training the above tokenized sentences with GloVe model. 

In [9]:
import pickle
with open ('words.txt', 'rb') as fp:
    wordlist = pickle.load(fp)
with open('embeddings.txt','rb') as fp:
    embeddings = pickle.load(fp)

We can see that the word if for the "UNKNOWN_TOKEN" is 11 in the wordlist. In other words, the index is 11 for the unknown token

In [10]:
word_to_index = dict([(w,i) for i,w in enumerate(wordlist)])
word_to_index['UNKNOWN_TOKEN']

11

Now, it is time to turn the words in the tokenized sentences to integers, typially these are the index of the words from the wordlist. Why are we doing this is, we need words in vector form in order to feed the LSTM network. 

To do that, first we replace the words with the indices and form sentences (Shown below) then convert the indices into a vector with a lookup table from the embeddings we have got using GloVe.  

In [11]:
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]

In [12]:
import numpy as np
X_train = np.asarray([[word_to_index[w] for w in sent] for sent in tokenized_sentences])

The below couple of cells shows what I have discussed above. tokenized_sentences[1] is the sentence which is converted in to a sentences with indices, shown as the output from X_train[1]

In [13]:
X_train[1]

[1, 4, 340, 1695, 3, 156, 774, 11, 2, 0]

In [14]:
tokenized_sentences[1]

['SENTENCE_START',
 'what',
 'films',
 'featured',
 'the',
 'character',
 'popeye',
 'UNKNOWN_TOKEN',
 '?',
 'SENTENCE_END']

There are around 1473 sentences and all the sentences have variable lenghts. Our static LSTM network expects inputs to be of fixed dimension. So, in order to even out the length of different sentences, we are padding each sentence equal to that of a sentence which has maximum lenght in the courpus. 

The max length is found out to be 35. So, all the sentences after padding should have a lenght of 35. 

In [15]:
import tensorflow as tf
import numpy as np
max_length = max(len(row) for row in X_train)
x_padded = np.array([row + [0] * (max_length - len(row)) for row in X_train])

In [16]:
targets = [0]*5

We have five classes namely, who, what , when, affirmation, unknown. We also need these objects in the form of vectors. I am converting each object in class to vector in the below format. 

In [17]:
# who, what, when, affirmation, unknown
label = []
p =q =r=s=t = 0
for i in labels:
    if i == 'who' : 
        label.append([1,0,0,0,0])
        p += 1
    elif i == 'what' : 
        label.append([0,1,0,0,0])
        q += 1
    elif i == 'when' : 
        label.append([0,0,1,0,0])
        r += 1
    elif i == 'affirmation' : 
        label.append([0,0,0,1,0])
        s += 1
    else : 
        label.append([0,0,0,0,1])
        t += 1

As I said at the start, label have the classes of the sentences, label[100] is the class of 100th sentences in the x_padded variable i.e., x_padded[100] 

We see, 100th sentence has the class 'who, 200th sentence has the class 'unknown', and 300th sentence has the class 'what'.

In [18]:
label[100], label[200], label[300]

([1, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 1, 0, 0, 0])

We see that there are 401 number of sentences that belong who class and so forth.  

In [19]:
print("Who:", p, 'when:', q, 'what:', r, 'affirmation:', s, 'unknown:', t)

Who: 401 when: 607 what: 96 affirmation: 104 unknown: 275


In [20]:
labels[100], labels[200], labels[300]

('who', 'unknown', 'what')

There are 1473 sentences in total in the dataset. I wanted to use a batch size of 50 and also, 50 sentences for testing. 

So, I made the first 1400 sentences available for training purposes and the last 50 for testing as shown below

In [21]:
x_train = x_padded[:1400]
y_train = label[:1400]
x_test = x_padded[1400:1450]
y_test = label[1400:1450]
len(x_train),len(y_train),len(x_test),len(y_test)

(1400, 1400, 50, 50)

In [22]:
len(x_train[0]),len(y_train[0])

(35, 5)

### Model 1 - Basic LSTM

The below is the model that we are going to use for training. I have used a basic LSTM Cell for training. I have commented in the code whereever a clarification is needed . 

In [30]:
tf.reset_default_graph()
from tensorflow.contrib import rnn 

num_epochs = 50    # no of times to run through the whole datset
batch_size = 50    # 50 sentences at a time are sent for training
num_steps = 35     # the lenght of each sentence is 35
state_size = 200   # the size of each word vector that we have got from the Glove model is 200. 
vocab_size = 3500  # out of 3700 odd unique words, we have used only 3500 words as vocab size.
num_classes = 5    # we have 5 classes 

# Placeholders
x = tf.placeholder(tf.int32, [batch_size, None])
y = tf.placeholder(tf.int32, [batch_size, None])

def train_neural_network(x): 
    
        def next_batch(step):   # get the next batch at each iteration 
            p,q = x_train[batch_size*step:batch_size*(step+1)], y_train[batch_size*step:batch_size*(step+1)]
            return p,q
        
        def test_batch(stp):    # gets the test batch
            a,b = x_test[batch_size*stp:batch_size*(stp+1)], y_test[batch_size*stp:batch_size*(stp+1)]
            return a,b
              
        def lstm_neural_network(x):  # the actual LSTM model
            
            # Embedding layer, this where each word is converted in to a word vector. 
            rnn_inputs = tf.nn.embedding_lookup(embeddings, x)
            print("This is the input for LSTM:", rnn_inputs)
            rnn_inputs = tf.transpose(rnn_inputs, [1,0,2])
            rnn_inputs = tf.reshape(rnn_inputs, [-1, state_size])
            rnn_inputs = tf.split(rnn_inputs, num_steps, 0)
            print("This the input after Processing the dimensions for LSTM:", rnn_inputs)
            
            # LSTM Cell used for training the model 
            cell = rnn.BasicLSTMCell(state_size) 
            rnn_outputs, rnn_states = rnn.static_rnn(cell, rnn_inputs, dtype=tf.float32)
            
            # the weight variable 
            W = tf.get_variable("W", [state_size, num_classes], dtype=tf.float32)
            
            # the bias variable
            b = tf.get_variable("b", [num_classes], dtype=tf.float32)
            
            # outputs from the LSTM model without softmax applied. 
            
            logits = tf.matmul(rnn_outputs[-1], W) + b
            
            # when softmax is applied for the logits, we get the probabilities for each class. can be used directly in testing. 
            probs = tf.nn.softmax(logits)
            return logits, probs

        prediction, probability  = lstm_neural_network(x)
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))  # cal loss
        optimizer = tf.train.AdamOptimizer().minimize(cost)   # learning rate 0.01 by default. 
        saver = tf.train.Saver()
        
        with tf.Session() as sess:
            sess.run(tf.initialize_all_variables())
            for epoch in range(num_epochs):
                epoch_loss = 0 
                for step in range(int(len(x_train)/batch_size)):
                    epoch_x, epoch_y = next_batch(step)
                    _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                    epoch_loss += c
                print('Epoch', epoch, 'completed out of',num_epochs,'loss:',epoch_loss)
            save_path = saver.save(sess, "./modell/")
            print("Model saved in file: %s" % save_path)
            
            for stp in range(int(len(x_test)/batch_size)):
                s,u = test_batch(stp)
                predictedclass = np.argmax(probability.eval({x: s, y: u}),1)
                actualclass = np.argmax(y.eval({x: s, y: u}),1)
                lbl = ['who','what','when','affirmation','unknown']
                for i,j in enumerate(X_train[1400:1450]):
                    print("Test Sentence:\n%s" % (' '.join([wordlist[x] for x in j])))
                    print("Predicted Class:", lbl[predictedclass[i]],  "\nActual Class:", lbl[actualclass[i]])

In [31]:
train_neural_network(x)

This is inputs: Tensor("embedding_lookup:0", shape=(50, ?, 200), dtype=float32)
This: [<tf.Tensor 'split:0' shape=(?, 200) dtype=float32>, <tf.Tensor 'split:1' shape=(?, 200) dtype=float32>, <tf.Tensor 'split:2' shape=(?, 200) dtype=float32>, <tf.Tensor 'split:3' shape=(?, 200) dtype=float32>, <tf.Tensor 'split:4' shape=(?, 200) dtype=float32>, <tf.Tensor 'split:5' shape=(?, 200) dtype=float32>, <tf.Tensor 'split:6' shape=(?, 200) dtype=float32>, <tf.Tensor 'split:7' shape=(?, 200) dtype=float32>, <tf.Tensor 'split:8' shape=(?, 200) dtype=float32>, <tf.Tensor 'split:9' shape=(?, 200) dtype=float32>, <tf.Tensor 'split:10' shape=(?, 200) dtype=float32>, <tf.Tensor 'split:11' shape=(?, 200) dtype=float32>, <tf.Tensor 'split:12' shape=(?, 200) dtype=float32>, <tf.Tensor 'split:13' shape=(?, 200) dtype=float32>, <tf.Tensor 'split:14' shape=(?, 200) dtype=float32>, <tf.Tensor 'split:15' shape=(?, 200) dtype=float32>, <tf.Tensor 'split:16' shape=(?, 200) dtype=float32>, <tf.Tensor 'split:17' 

### Model 2 - Dynamic LSTM

The below code uses a dynamic LSTM which is unlike the above static LSTM. We have padded each sentence to 35 in the above and also, we are feeding the same to the LSTM model. Here, in the dynamic model, although we do the same, in addition we have control over the model to tell it the actual length of each sentence so that while training, it would only concentrate on the real length of the sentence and leaves/ just copies the values of rest of the padded length in the outputs. 

In [32]:
tf.reset_default_graph()
from tensorflow.contrib import rnn 

num_epochs = 100
batch_size = 50
num_steps = 35
state_size = 200
vocab_size = 3500
num_classes = 5

# Placeholders
x = tf.placeholder(tf.int32, [batch_size, None])
seqlen = tf.placeholder(tf.int32, [batch_size])
y = tf.placeholder(tf.int32, [batch_size, None])
keep_prob = tf.constant(1.0)
sequencelength = [len(i) for i in X_train][:1400]
testsequencelength = [len(i) for i in X_train][1400:1450]

def train_neural_network(x): 
    
        def next_batch(step):
            p,q = x_train[batch_size*step:batch_size*(step+1)], y_train[batch_size*step:batch_size*(step+1)]
            r = sequencelength[batch_size*step:batch_size*(step+1)]
            return p,q,r
        
        def test_batch(stp):    
            a,b = x_test[batch_size*stp:batch_size*(stp+1)], y_test[batch_size*stp:batch_size*(stp+1)]
            c = testsequencelength[batch_size*stp:batch_size*(stp+1)]
            return a,b,c
              
        def lstm_neural_network(x):
            # Embedding layer
            rnn_inputs = tf.nn.embedding_lookup(embeddings, x)
            print(rnn_inputs)
            # RNN
            initial_state = tf.tile(tf.get_variable('initial_state', [1, state_size],
                                    initializer=tf.constant_initializer(0.0)),[batch_size,1])
            #inputs = tf.unstack(rnn_inputs, num=num_steps, axis=1)   
            cell = tf.contrib.rnn.GRUCell(state_size)
            #cell = tf.contrib.rnn.DropoutWrapper(cell=cell, output_keep_prob = 0.5)
            print("This is RNN going in for LSTM network:", rnn_inputs)
            print("This is the Sequence Length:", seqlen)
            rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, rnn_inputs, sequence_length=seqlen,
                                                                   initial_state=initial_state)

            rnn_outputs = tf.transpose(rnn_outputs, [1,0,2])
            print("This is the Output from the model:",rnn_outputs)
            W = tf.get_variable("W", [state_size, num_classes], dtype=tf.float32)
            b = tf.get_variable("b", [num_classes], dtype=tf.float32)
            logits = tf.matmul(rnn_outputs[-1], W) + b
            probs = tf.nn.softmax(logits)
            return logits, probs

        prediction, probability  = lstm_neural_network(x)
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))
        optimizer = tf.train.AdamOptimizer().minimize(cost)   
        saver = tf.train.Saver()
        
        with tf.Session() as sess:
            sess.run(tf.initialize_all_variables())
            for epoch in range(num_epochs):
                epoch_loss = 0 
                for step in range(int(len(x_train)/batch_size)):
                    epoch_x, epoch_y, sl = next_batch(step)
                    _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y, seqlen: sl})
                    epoch_loss += c
                print('Epoch', epoch, 'completed out of',num_epochs,'loss:',epoch_loss)
            save_path = saver.save(sess, "./model/")
            print("Model saved in file: %s" % save_path)
            
            for stp in range(int(len(x_test)/batch_size)):
                s,u,v = test_batch(stp)
                predictedclass = np.argmax(probability.eval({x: s, y: u,seqlen: v}),1)
                actualclass = np.argmax(y.eval({x: s, y: u,seqlen: v}),1)
                lbl = ['who','what','when','affirmation','unknown']
                for i,j in enumerate(X_train[1400:1450]):
                    print("Test Sentence:\n%s" % (' '.join([wordlist[x] for x in j])))
                    print("Predicted Class:", lbl[predictedclass[i]],  "\nActual Class:", lbl[actualclass[i]])

In [33]:
train_neural_network(x)

Tensor("embedding_lookup:0", shape=(50, ?, 200), dtype=float32)
this is rnn going in: Tensor("embedding_lookup:0", shape=(50, ?, 200), dtype=float32)
this is the seqlen: Tensor("Placeholder_1:0", shape=(50,), dtype=int32)
this is the output: Tensor("transpose_1:0", shape=(?, 50, 200), dtype=float32)
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch 0 completed out of 100 loss: 52.8493211269
Epoch 1 completed out of 100 loss: 52.2709000111
Epoch 2 completed out of 100 loss: 51.7876154184
Epoch 3 completed out of 100 loss: 51.3163787127
Epoch 4 completed out of 100 loss: 50.8565140963
Epoch 5 completed out of 100 loss: 50.4082621336
Epoch 6 completed out of 100 loss: 49.9716858864
Epoch 7 completed out of 100 loss: 49.5467250347
Epoch 8 completed out of 100 loss: 49.1332461834
Epoch 9 completed out of 100 loss: 48.7310907841
Epoch 10 completed out of 100 loss: 48.3400743008
Epoch 11 completed out of 100 loss: 47.9600012302
Epoch 12 completed out of 100 loss:

### Bottom Line

The Basic LSTM model (#1) does a good job predicting accurately most of the time (as in model 1), 42 sentences are classified as correct out of 50 sentences. 

The other model (#2) doesn't show convincing results. However, the model can be improved by using multiple layers, adding a dropout layer, or amneding the padded sentences. 

We could otherwise implement a Convolutoinal Neural Network to see how the model predicts. As we are interested in the sentence classification , and CNN are best known for classification problems because there the intention is to extract features from the vectors, this problem , I reckon, does better on a CNN model. 

Also, we are not interested in the syntactic form in the sentence (since we are not generating any sentences), so more or less, LSTM could be used but CNN should outperform the current model. 

I did not had enough time to work further on this model, I will implement the CNN model and share the results whenever I find sometime. 